# Data Migration

If you want to use SQLite instead of MySQL, uncomment out the following cell and comment out the two cells after it.

In [ ]:
# import sqlite3
# import pandas as pd


# DATABASE_NAME = 'databaseSQLite.db'
# FILE_NAME = 'data_modified.xlsx'

# # Connect to a database
# conn = sqlite3.connect(DATABASE_NAME)

# #Create a cursor object
# db = conn.cursor()

Before running, it is recommended to create a conda environment. 

Using the env.yml file attached,

run the command in terminal : conda env create -f env.yml

When running the cells below, make sure the kernel used is migrationEnv.

In [1]:
# Environment
# Connection between python and mySQL 
# https://dev.mysql.com/doc/connector-python/en/
import mysql.connector
import pandas as pd

### Preparation

In [80]:
# Make sure you have created an empty database in mysql
DATABASE_NAME = 'FILL IN'
FILE_NAME = 'data_modified.xlsx'
DB_PASSWORD = 'FILL IN'
# Connect to a database
conn = mysql.connector.connect(host='localhost',user='root',password=DB_PASSWORD,database=DATABASE_NAME)

# Create a cursor object
db = conn.cursor()

### Create tables

In [60]:
# DB Setup
# Create all the required tables in the database
# Other option would be to go to the mysql terminal and follow the queries as in dbmysql.txt. If you follow this step, don't run this cell. 

def create_tables(db):

    # Create tank
    db.execute('''CREATE TABLE Tank(
        Name VARCHAR(100) NOT NULL,
        PRIMARY KEY(Name)
    );''')

    # NB
    ## AUTO_INCREMENT Not supported by SQLite 
    ## Changing it for the rest of the tables
    ## changed from: Id INT unsigned NOT NULL AUTO_INCREMENT
    ## changed to: Id INTEGER PRIMARY KEY
    ## Due to this Primary key is identified twice
    ## Removing line PRIMARY KEY (Id) from every statement

    # Create Rack
    db.execute('''CREATE TABLE Rack(
    Id INT unsigned NOT NULL AUTO_INCREMENT,
    Name INT NOT NULL,
    Tank VARCHAR(100) NOT NULL,
    PRIMARY KEY (Id),
    FOREIGN KEY (Tank) REFERENCES Tank(Name)
    );''')

    # Create box
    db.execute('''CREATE TABLE Box(
    Id INT unsigned NOT NULL AUTO_INCREMENT,
    Number INT NOT NULL, 
    Rack_id INT unsigned NOT NULL,
    PRIMARY KEY (Id),
    FOREIGN KEY (Rack_id) REFERENCES Rack(Id)
    );''')

    # Create Location
    db.execute('''CREATE TABLE Location(
    Id INT unsigned NOT NULL AUTO_INCREMENT,
    Name VARCHAR(5) NOT NULL, 
    Box_id INT unsigned NOT NULL,
    PRIMARY KEY (Id),
    FOREIGN KEY (Box_id) REFERENCES Box(Id)
    );''') 
    
    # Create Vial
    db.execute('''CREATE TABLE Vial(
    Id INT unsigned NOT NULL AUTO_INCREMENT,
    Location_id INT unsigned NOT NULL, 
    Line VARCHAR(75),
    Passage VARCHAR(75),
    Origin VARCHAR(75),
    Type VARCHAR(75),
    Subtype VARCHAR(75),
    Pool VARCHAR(75),
    CRISPR_EDIT VARCHAR(75),
    Genotype VARCHAR(75),
    Reprogramming_method VARCHAR(75),
    Age INT,
    Gender VARCHAR(75),
    Info VARCHAR(75),
    Media VARCHAR(75),
    ECM VARCHAR(75),
    Date VARCHAR(75),
    Initials VARCHAR(75),
    Cell_numbers VARCHAR(75),
    Confluency INT,
    PRIMARY KEY (Id),   
    FOREIGN KEY (Location_id) REFERENCES Location(Id)
    );''') 

    # Create Note
    db.execute('''CREATE TABLE Note(
    Id INT unsigned NOT NULL AUTO_INCREMENT,
    Note VARCHAR(500),
    Vial_id INT unsigned NOT NULL,
    PRIMARY KEY (Id),
    FOREIGN KEY (Vial_id) REFERENCES Vial(Id)
    );''') 



In [61]:

#Execute this to generate tables for sqlite db
create_tables(db)
conn.commit()


### Read data

In [62]:
# Open the excel file
xl = pd.ExcelFile(FILE_NAME)


In [63]:
# Read Tank 4 Sheet
# There are 15 sections each containing 26 rows including the header
# Sections are separated by a single empty row. (see changes.txt)
dfs_T4 = []
for i in range(15):
    skiprows = 27*i 
    df = xl.parse(sheet_name=0, nrows=26, skiprows=skiprows)
    dfs_T4.append(df)

In [64]:
# Read Sheet NC Tank BMI 2nd floor
# There is only one section containing 82 rows including the header
df_NC = xl.parse(sheet_name=1)

In [65]:
# Read Glacier BM2 P-floor
# There are 11 sections each containing 82 rows including the header
# Sections are separated by a single empty row. (see changes.txt)
dfs_G = []
for i in range(11):
    skiprows = 83*i 
    df = xl.parse(sheet_name=2, nrows=82, skiprows=skiprows)
    dfs_G.append(df)

In [66]:
# Close file
xl.close()

## Populate

### Part 1

The following cells generate dataframes for all tables except Vial and Note.

Incremental ids are attached to dataframes for convienence. 

The column names of dataframe correspond to column names in the database schema

In [67]:
# Generate dataframe for table Tank.
# There are 3 values Tank4, NCTank and Glacier
TANKS = ['Tank4', 'NCTank', 'Glacier']
TANKS_DF = pd.DataFrame({'Name': TANKS})

TANKS_DF

,Name
0,Tank4
1,NCTank
2,Glacier


In [68]:
# Generate dataframe for table Rack
# Tank4 has racks: 4,5,6
# NCTank has racks: 12
# Glacier has racks: 1
# Incremental Id column attached to dataframe
RACKS = {TANKS[0]:[4,5,6],
         TANKS[1]:[12], #confirm
         TANKS[2]:[1]}

rack_data = [{'Name': rack, 'Tank': tank} for tank, racks in RACKS.items() for rack in racks]

RACKS_DF = pd.DataFrame(rack_data)
RACKS_DF['Id'] = range(1, len(RACKS_DF) + 1)
RACKS_DF

,Name,Tank,Id
0,4,Tank4,1
1,5,Tank4,2
2,6,Tank4,3
3,12,NCTank,4
4,1,Glacier,5


In [13]:
# Generate dataframe for table Box
# [Tank4]   For Rack Ids = [1,2,3] boxes are 5 in total
# [NCTank]  For Rack Ids = [4] boxes are 1 in total (numbered 6)
# [Glacier] For Rack Ids = [5] boxes are 11 in total
# Incremental Id column attached to dataframe
BOXES = {'1':range(1,6),
         '2':range(1,6), 
         '3':range(1,6),
         '4':[6], #confirm
         '5':range(1,12)}
box_data = [{'Number': box, 'Rack_id': int(rack)} for rack, boxes in BOXES.items() for box in boxes]

BOXES_DF = pd.DataFrame(box_data)
BOXES_DF['Id'] = range(1, len(BOXES_DF) + 1)
BOXES_DF

,Number,Rack_id,Id
0,1,1,1
1,2,1,2
2,3,1,3
3,4,1,4
4,5,1,5
5,1,2,6
6,2,2,7
7,3,2,8
8,4,2,9
9,5,2,10


In [69]:
# Generate dataframe for table Location 
# [Tank4] For Box_id [1-15] locations are named from 1-25
# [NCTank,Glacier] For Box_id [16-27] locations are named A1-I9
# Incremental Id column attached to dataframe

array_list = list(range(1,26))
strings_list = [f"{char}{num}" for char in 'ABCDEFGHI' for num in range(1, 10)]

loc_data_1 = [{'Name': name, 'Box_id': box} for box in range(1,16) for name in array_list]
loc_data_2 = [{'Name': name, 'Box_id': box} for box in range(16,28) for name in strings_list]
temp_df1 = pd.DataFrame(loc_data_1)
temp_df2 = pd.DataFrame(loc_data_2)
LOC_DF = pd.concat([temp_df1, temp_df2], ignore_index=True)
LOC_DF['Id'] = range(1, len(LOC_DF) + 1)
LOC_DF

,Name,Box_id,Id
0,1,1,1
1,2,1,2
2,3,1,3
3,4,1,4
4,5,1,5
...,...,...,...
1342,I5,27,1343
1343,I6,27,1344
1344,I7,27,1345
1345,I8,27,1346


Generate sql insert queries and execute

In [70]:
# The function generates sql insert queries given the dataframe (SOURCE) and table name (TARGET)
def SQL_INSERT_STATEMENT_FROM_DATAFRAME(SOURCE, TARGET):
    sql_texts = []
    for index, row in SOURCE.iterrows():
        columns = ', '.join(SOURCE.columns)
        values = ', '.join(f"'{value}'" if SOURCE[col].dtype == 'object' else str(value) for col, value in row.items())
        sql_texts.append(f"INSERT INTO {TARGET} ({columns}) VALUES ({values})")
    return sql_texts

In [71]:
# For every dataframe, generate sql insert queries and execute

dic = {'Tank':TANKS_DF,
       'Rack':RACKS_DF,
       'Box':BOXES_DF,
       'Location':LOC_DF
       }

for table,df in dic.items():
    queries = SQL_INSERT_STATEMENT_FROM_DATAFRAME(df,table)

    for query in queries:

        db.execute(query)
        

conn.commit()

### Part 2

In [72]:
# The function returns sql insert queries for table vial and notes array
def SQL_INSERT_STATEMENT_FOR_VIAL_NOTE(SOURCE,TARGET,  box_number, rack_name, tank_name):
    
    # Rename Reprogramming_method, Cell_numbers
    SOURCE = SOURCE.rename(columns={'Reprogramming method': 'Reprogramming_method', 'Cell number': 'Cell_numbers'})
 
    # separate notes
    NOTES = SOURCE[['Notes']]
    SOURCE.drop(columns=['Notes',"Media/ECM"], inplace=True) # TODO dropping column Media/ECM for now # Split Media/ECM ??
    
    vial_sql_texts = []
    notes_array = []
    for index, row in SOURCE.iterrows():
        columns = ', '.join(SOURCE.columns[1:])
        location_name = row.iloc[0]
        values = ', '.join(f"'{value}'" if SOURCE[col].dtype == 'object' else str(value) for col, value in row[1:].items())
        insert_query = f"""INSERT INTO {TARGET} (Location_id, {columns}) \
        VALUES ( \
            (SELECT Location.Id \
            FROM Location \
            INNER JOIN Box ON Location.Box_id = Box.Id \
            INNER JOIN Rack ON Box.Rack_id = Rack.Id \
            INNER JOIN Tank ON Rack.Tank = Tank.Name \
            WHERE Location.Name = '{location_name}' \
            AND Box.Number = {box_number} \
            AND Rack.Name = {rack_name} \
            AND Tank.Name = '{tank_name}' \
            ), \
            {values}\
        );\
        """
        # Replace nans with null
        insert_query = insert_query.replace("'nan'",'null').replace('nan','null')
        
        vial_sql_texts.append(insert_query)

        # Split and store notes based on semicolon delimiter
        note = NOTES['Notes'].iloc[index]
        notes_array.append(str(note).split(';') if not pd.isnull(note) else []) 

        
    return vial_sql_texts, notes_array 

In [74]:
# The function execute an insert query for table vial, retrives the last row id of vial
# and stores the relevant notes in note table
def populate_vial_table(SOURCE, box_number, rack_name, tank_name):

    vial_sql_texts, notes = SQL_INSERT_STATEMENT_FOR_VIAL_NOTE(SOURCE,
                                                               TARGET='Vial',
                                                               box_number=box_number,
                                                               rack_name=rack_name,
                                                               tank_name=tank_name)
    
    for index, query in enumerate(vial_sql_texts):
        db.execute(query)
        #print(query)

        if (len(notes[index])):
            Vial_id = db.lastrowid

            for note in notes[index]:

                insert_query = f"INSERT INTO Note (Note, Vial_id) VALUES ('{str(note).strip()}', {Vial_id});"
                db.execute(insert_query)

    conn.commit()

Populate Vials along with notes

In [75]:
# Populate Tank 4
index = 0
for rack in [4,5,6]:
    for box in range(1,6):

        populate_vial_table(dfs_T4[index],
                            box_number=box,
                            rack_name=rack,
                            tank_name=TANKS[0])


In [76]:
# Populate NC Tank
populate_vial_table(df_NC,
                    box_number=6,
                    rack_name=12,
                    tank_name=TANKS[1])

In [77]:
# Populate Glacier
for box in range(1,12):
    populate_vial_table(dfs_G[box-1],
                    box_number=box,
                    rack_name=1,
                    tank_name=TANKS[2])

In [78]:
conn.close()